In [1]:
from reconciler import reconcile
import pandas as pd


original_data = "../../analysis/data/panglaodb/"
results_path = "../../analysis/results/true_matches/"

original_cells_organs = pd.read_csv(f"{original_data}cells_organs_germlayers.csv")
original_cells_organs.head()

,cell_type,germ_layer,organ
0,Purkinje neurons,Ectoderm,Brain
1,Schwann cells,Ectoderm,Brain
2,Oligodendrocytes,Ectoderm,Brain
3,Microglia,Ectoderm,Brain
4,Astrocytes,Ectoderm,Brain


In [2]:
cell_types = original_cells_organs["cell_type"]
# Reconcile against cell types on Wikidata
reconciled = reconcile(cell_types, type_id="Q189118")

100%|██████████| 21/21 [02:07<00:00,  6.05s/it]


Now I`ll check the matches labeled as "true" for false positives.

In [15]:
cell_type_matches_considered_true = reconciled[reconciled["match"] == True]
cell_type_matches_considered_true.to_csv("../results/cell_type_matches_considered_true_29_10_2020.csv")

I have saved the file as a csv and opened on LibreOffice. I am looking at each match for inconsistencies and will note down any inconsistencies found.

- Dopaminergic neuron is matched to "dopaminergic cell group", and the item is a bit weird, it uses info from the Foundational Model of Anatomy (FMA). There is an item for "dopaminergic neuron" on Wikitada (Q66591220) with a different FMA identifier. Arguably is a better match, so I`ll use that instead. 


Modifications to make:

- Dopaminergic neurons: Q66591220


I`ll prepare a verified dict for name to qid reconciliation of cell types.



In [16]:
panglao_cell_type_to_qid = {}

for i, row in cell_type_matches_considered_true.iterrows():
    name = row["input_value"]
    qid = row["id"]
    
    panglao_cell_type_to_qid[name] = qid

# Fix manually false positive match

panglao_cell_type_to_qid["Dopaminergic neurons"] = "Q66591220"

Nice. Now let`s look at the matches that were found, but considered false. 

In [29]:
cell_type_matches_considered_false = reconciled[reconciled["match"] == False]
cell_type_matches_considered_false = cell_type_matches_considered_false[cell_type_matches_considered_false["id"].notna()]

cell_type_matches_considered_false.to_csv("../results/cell_type_matches_considered_false_29_10_2020.csv")

Once again I open it with LibreOffice. Now I will have to correc most of them. 

- Perisynaptic Schwann cells (Q17157019) seems to be too specific. Schwann cell (Q465621) is the better match. 
- radial glial cell (Q4387286) is similar, but different of Astrocyte progenitor cells. I have created an item for astrocyte progenitor cells (Q101001053).
- Adaptive NK cells (Q49000020) is too specific. natural killer cell (Q332181) is the better match. 
- Endoplasmic Reticulum Stress in Beta Cells (Q5376385) is just a topic. beta cell (Q1767180) is the better match.
- Perineuronal satellite cells (Q70068158) is a wrong match. Satellite cells in PanglaoDB are "precursors to skeletal muscle cells, able to give rise to satellite cells or differentiated skeletal muscle cells." There is a "Satellite cell (Q66592694)" but I am not sure  if that is what we want. I`ll create a new term.
- subependymal glioma (Q18556497) is a wrong match for "Ependymal cells." ependymal cell (Q28646122) is the better match.
- cardiomyocyte (Q1047227) is too specific for "Myocytes". myocyte (Q428914) is a better match. 
- natural killer T cell (Q224930) is similar, but different from "T cells". T-lymphocytes (Q193529) is a better match. 



As there is a large number of mismatches, I`ll plainly add the ones that need modification to the list below. I`ll note any case that is different from the ones above and any creation of new 





Modifications to make:

- Schwann cells: Q465621
- Astrocyte progenitor cells: Q101001053
- NK cells: Q332181
- Beta cells: Q1767180
- Satellite cells: Q101001061
- Ependymal cells: Q28646122
- Myocytes: Q428914
- T cells: Q193529



